In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resume-dataset/Resume/Resume.csv
/kaggle/input/resume-dataset/data/data/DESIGNER/22506245.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/13998435.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/67582956.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/34349255.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/26790545.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/12674307.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/11807040.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/44145704.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/27497542.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/11155153.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/17555081.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/41506705.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/54201930.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/24583187.pdf
/kaggle/input/resume-dataset/data/data/DESIGNER/38565119.pdf
/kaggle/input/resume-dataset/data/data

# Cell 1: Import Libraries



In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Download stopwords if not already available
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Cell 2: Load the Dataset


In [10]:
# Adjust the path as needed based on how your files are organized in Kaggle
df = pd.read_csv('/kaggle/input/resume-dataset/Resume/Resume.csv')  # Example path
df.head()


,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


# Cell 3: Inspect the Data


In [11]:
print("DataFrame Shape:", df.shape)
print("Columns:", df.columns)
print("Null Values:\n", df.isnull().sum())
df.sample(5)


DataFrame Shape: (2484, 4)
Columns: Index(['ID', 'Resume_str', 'Resume_html', 'Category'], dtype='object')
Null Values:
 ID             0
Resume_str     0
Resume_html    0
Category       0
dtype: int64


,ID,Resume_str,Resume_html,Category
1707,28505854,ENGINEERING MANAGER Summary...,"<div class=""fontsize fontface vmargins hmargin...",ENGINEERING
102,10399912,HR PERSONNEL ASSISTANT Summar...,"<div class=""fontsize fontface vmargins hmargin...",HR
663,17597372,BUSINESS DEVELOPMENT CONSULTANT ...,"<div class=""fontsize fontface vmargins hmargin...",BUSINESS-DEVELOPMENT
1971,51638201,CONSTRUCTION MANGER III Summa...,"<div class=""fontsize fontface vmargins hmargin...",CONSTRUCTION
464,29926588,VOLUNTEER ADVOCATE Summary ...,"<div class=""fontsize fontface vmargins hmargin...",ADVOCATE


# Cell 4: Define a Text Cleaning Function


In [12]:
def clean_text(text):
    # 1. Remove HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # 2. Convert to lowercase
    text = text.lower()
    
    # 3. Remove punctuation and numbers (optional: you can refine regex as needed)
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # 4. Tokenize
    tokens = word_tokenize(text)
    
    # 5. Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
    
    # 6. Rejoin
    cleaned_text = ' '.join(tokens)
    return cleaned_text


# Cell 5: Apply Cleaning to the resume_str Column


In [13]:
df['cleaned_resume'] = df['Resume_str'].astype(str).apply(clean_text)
df[['Resume_str', 'cleaned_resume']].head()


,Resume_str,cleaned_resume
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,hr administrator marketing associate hr admini...
1,"HR SPECIALIST, US HR OPERATIONS ...",hr specialist us hr operations summary versati...
2,HR DIRECTOR Summary Over 2...,hr director summary years experience recruitin...
3,HR SPECIALIST Summary Dedica...,hr specialist summary dedicated driven dynamic...
4,HR MANAGER Skill Highlights ...,hr manager skill highlights hr skills hr depar...


# Cell 6: Encode the Target (Category)


In [14]:
# Factorize the categories (assign each unique category a numeric ID)
df['Category_encoded'] = pd.factorize(df['Category'])[0]
df[['Category', 'Category_encoded']].drop_duplicates().head(10)


,Category,Category_encoded
0,HR,0
110,DESIGNER,1
217,INFORMATION-TECHNOLOGY,2
337,TEACHER,3
439,ADVOCATE,4
557,BUSINESS-DEVELOPMENT,5
677,HEALTHCARE,6
792,FITNESS,7
909,AGRICULTURE,8
972,BPO,9


# Cell 7: Split the Data into Training and Test Sets

In [15]:
X = df['cleaned_resume']
y = df['Category_encoded']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # This ensures class distribution is balanced in splits
)

print("Train set size:", X_train.shape, y_train.shape)
print("Test set size:", X_test.shape, y_test.shape)


Train set size: (1987,) (1987,)
Test set size: (497,) (497,)


# Cell 8: Vectorize Text Using TF-IDF

In [16]:
tfidf = TfidfVectorizer(max_features=3000)  # You can adjust max_features as needed

# Fit on the training data, then transform both train and test
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

print("TF-IDF training shape:", X_train_tfidf.shape)
print("TF-IDF test shape:", X_test_tfidf.shape)


TF-IDF training shape: (1987, 3000)
TF-IDF test shape: (497, 3000)


# Cell 9: Train a Logistic Regression Model

In [17]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

print("Training completed!")


Training completed!


# Cell 10: Evaluate the Model

In [18]:
y_pred = model.predict(X_test_tfidf)

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Test Accuracy: 0.6398390342052314

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.82      0.77        22
           1       0.85      0.81      0.83        21
           2       0.56      0.83      0.67        24
           3       0.70      0.70      0.70        20
           4       0.38      0.46      0.42        24
           5       0.64      0.67      0.65        24
           6       0.48      0.57      0.52        23
           7       0.89      0.67      0.76        24
           8       0.75      0.23      0.35        13
           9       0.00      0.00      0.00         4
          10       0.47      0.65      0.55        23
          11       0.42      0.22      0.29        23
          12       0.68      0.79      0.73        19
          13       0.00      0.00      0.00         7
          14       0.92      0.92      0.92        24
          15       0.89      0.71      0.79        24
          16       0.44

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Cell 11: Calculate Additional Evaluation Metrics

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate the metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Evaluation Metrics:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")


Evaluation Metrics:
Accuracy : 0.6398
Precision: 0.6358
Recall   : 0.6398
F1 Score : 0.6264


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
